In [158]:
# import the libraries
import pandas as pd
import numpy as np
import sys

In [159]:
# read the data
df = pd.read_csv("../data/total_processed.csv", index_col=0)

In [160]:
df.columns

Index(['sha', 'message', 'comment_count', 'date', 'total_deletions',
       'total_additions', 'total', 'comments', 'additions', 'deletions',
       'changes', 'status', 'filenames', 'name', 'fullname', 'private', 'fork',
       'size', 'watchers_count', 'has_issues', 'has_downloads', 'has_wiki',
       'forks_count', 'open_issues_count', 'forks', 'open_issues', 'watchers',
       'network_count', 'type', 'admin', 'push', 'pull', 'label', 'owner',
       'language_C', 'language_C#', 'language_C++', 'language_CSS',
       'language_CoffeeScript', 'language_Go', 'language_Java',
       'language_JavaScript', 'language_PHP', 'language_Perl',
       'language_Python', 'language_R', 'language_Ruby', 'language_Scala',
       'language_Shell', 'language_TypeScript', 'total_files', 'results'],
      dtype='object')

### Mean of the commit label

In [161]:
def mean(items):
    index = 0
    result = []
    value = 0

    for item in items:
        index = index + 1

        if item == 1:
            value = value + 1

        result.append((value/index)*100)

    return result

In [162]:
# create a dictionary project_name + mean_values for the commit classification
projects = df['name'].unique()
commit_mean_dict = {}

for project in projects:
    commits = df[df.name==project].results
    mean_commits = mean(commits)
    commit_mean_dict[project] = mean_commits

In [163]:
# replace all the data
df['mean_commits'] = 0

for project in projects:
    df.loc[df['name'] == project, 'mean_commits'] = commit_mean_dict[project]

In [164]:
df[df['name' == 'stat-cookbook']].mean_commits

KeyError: False

In [29]:
def derive(items):
    i = 1
    result = []
    result.append(0)

    while i < len(items):
        if items[i-1] == 0:
            result.append((items[i]-items[i-1]))
            i = i + 1
        else:
            result.append((items[i]-items[i-1])/items[i-1])
            i = i + 1

    return result

In [30]:
for column in df.columns:
    col_values = derive(df[column].values)
    df[column+'_derivate'] = col_values

In [31]:
df = df.drop("commit_derivate", axis=1)

In [32]:
results = df.commit.values

In [33]:
df = df.drop("commit", axis=1)

In [34]:
df['commit'] = results 

In [35]:
df.commit.value_counts()

0    407761
1    193316
Name: commit, dtype: int64

In [ ]:
df.to_csv('commits_transformed_new.csv')